## Collect & Clean weather data for test_flights dataset

In [32]:
import pandas as pd
import numpy as np

In [33]:
import datetime as dt

In [34]:
import requests
import time

### Import table of unique date/city combinations
***** where was this table created?????? should clean that notebook********

This is to reduce the number of API pulls

In [5]:
flight_unique = pd.read_csv('data/flights_test_uniques.csv', index_col=0)

In [6]:
flight_unique.head()

,fl_date,city,count,weather_categ
758,2020-01-03,"Chicago, IL",2591,empty
408,2020-01-02,"Chicago, IL",2587,empty
1463,2020-01-05,"Chicago, IL",2556,empty
1813,2020-01-06,"Chicago, IL",2556,empty
2166,2020-01-07,"Chicago, IL",2413,empty


### Import lat long table
This table was created and cleaned in the EDA_Task3 notebook

In [100]:
#api requires lat/long, we found this table via google
latlong_df = pd.read_csv('data/uscities_ll.csv')

In [8]:
latlong_df.shape

(30409, 6)

In [9]:
latlong_df

,city,state_id,lat,lng,timezone,city_state
0,New York,NY,40.6943,-73.9249,America/New_York,"New York, NY"
1,Los Angeles,CA,34.1141,-118.4068,America/Los_Angeles,"Los Angeles, CA"
2,Chicago,IL,41.8375,-87.6866,America/Chicago,"Chicago, IL"
3,Miami,FL,25.7840,-80.2101,America/New_York,"Miami, FL"
4,Dallas,TX,32.7935,-96.7667,America/Chicago,"Dallas, TX"
...,...,...,...,...,...,...
30404,Drummond,ID,43.9996,-111.3433,America/Boise,"Drummond, ID"
30405,Lost Springs,WY,42.7652,-104.9255,America/Denver,"Lost Springs, WY"
30406,Provo,SD,43.1937,-103.8329,America/Denver,"Provo, SD"
30407,Goldcreek,MT,46.5838,-112.9284,America/Denver,"Goldcreek, MT"


In [101]:
#check for dupicate city_state combinations
latlong_df['city_state'].nunique()

30351

In [11]:
latlong_df = latlong_df.drop_duplicates(subset='city_state', keep="first")

In [12]:
latlong_df

,city,state_id,lat,lng,timezone,city_state
0,New York,NY,40.6943,-73.9249,America/New_York,"New York, NY"
1,Los Angeles,CA,34.1141,-118.4068,America/Los_Angeles,"Los Angeles, CA"
2,Chicago,IL,41.8375,-87.6866,America/Chicago,"Chicago, IL"
3,Miami,FL,25.7840,-80.2101,America/New_York,"Miami, FL"
4,Dallas,TX,32.7935,-96.7667,America/Chicago,"Dallas, TX"
...,...,...,...,...,...,...
30404,Drummond,ID,43.9996,-111.3433,America/Boise,"Drummond, ID"
30405,Lost Springs,WY,42.7652,-104.9255,America/Denver,"Lost Springs, WY"
30406,Provo,SD,43.1937,-103.8329,America/Denver,"Provo, SD"
30407,Goldcreek,MT,46.5838,-112.9284,America/Denver,"Goldcreek, MT"


### API functions

************* THIS IS WHERE WE WOULD LOAD THE FUNCTIONS INSTEAD ****************

In [13]:
#open weather api function
def open_weather_api(lat, long, date, timezone = "America%2FChicago"):
    """
    function returns individual api calls by location and date. 
    """
    #timezone = America%2FChicago #timezone format, opportunity to fine tune this - doesn't work for some reason, I think formatting
    
    url = f"https://archive-api.open-meteo.com/v1/era5?latitude={lat}&longitude={long}&start_date={date}&end_date={date}&hourly=precipitation,rain,snowfall,cloudcover&daily=precipitation_sum,rain_sum,snowfall_sum,precipitation_hours&timezone={timezone}"
    response = requests.get(url)
    return response.json()

In [14]:
test_api = open_weather_api("41.75","-87.75","2018-10-25", "America%2FNew_York")
test_api

{'latitude': 41.75,
 'longitude': -87.75,
 'generationtime_ms': 0.4220008850097656,
 'utc_offset_seconds': -14400,
 'timezone': 'America/New_York',
 'timezone_abbreviation': 'EDT',
 'elevation': 192.0,
 'hourly_units': {'time': 'iso8601',
  'precipitation': 'mm',
  'rain': 'mm',
  'snowfall': 'cm',
  'cloudcover': '%'},
 'hourly': {'time': ['2018-10-25T00:00',
   '2018-10-25T01:00',
   '2018-10-25T02:00',
   '2018-10-25T03:00',
   '2018-10-25T04:00',
   '2018-10-25T05:00',
   '2018-10-25T06:00',
   '2018-10-25T07:00',
   '2018-10-25T08:00',
   '2018-10-25T09:00',
   '2018-10-25T10:00',
   '2018-10-25T11:00',
   '2018-10-25T12:00',
   '2018-10-25T13:00',
   '2018-10-25T14:00',
   '2018-10-25T15:00',
   '2018-10-25T16:00',
   '2018-10-25T17:00',
   '2018-10-25T18:00',
   '2018-10-25T19:00',
   '2018-10-25T20:00',
   '2018-10-25T21:00',
   '2018-10-25T22:00',
   '2018-10-25T23:00'],
  'precipitation': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
  

In [16]:
def city_scrubber(city_str):
    """
    city name will be scrubbed to remove multiple names
    Dallas/Fort Something, TX -----> Dallas, TX
    """
    try:
        state = city_str.split(',')       #split city(s) and state
        cities = state[0].split('/')      #split the list of cities by /
        return cities[0] + ',' + state[1]
    except: 
        return "no state in your city string"

In [17]:
#test
city_scrubber("Dallas/Fort Something, TX")

'Dallas, TX'

In [18]:
city_scrubber("Austin, TX")

'Austin, TX'

In [19]:
city_scrubber("Austin")

'no state in your city string'

In [20]:
#function that returns the weather condition for a date/city
def weather_condition(city, date, lat_long_df):
    """
    returns the weather condition for a single date/city. 
    city format "city, ST"
    date format "2020-01-01"
    """
    #scrub city
    #print(city)
    city_scrub = city_scrubber(city)
    if city_scrub == 'no state in your city string':
        return city_scrub, city
    
    #pull the lat and long for city
    lat = None
    long = None
    try:
        lat = float(lat_long_df['lat'][lat_long_df['city_state'] == city_scrub])
        long = float(lat_long_df['lng'][lat_long_df['city_state'] == city_scrub])
    except:
        print(lat, long, city)
        pass
    
    #call the weather api
    if lat and long:         #none is False
        json_result = open_weather_api(lat, long, date)
        #print(json_result)
        try:
            total_rain = int(json_result['daily']['rain_sum'][0])                  # mm
        except: 
            total_rain = None
        try:
            total_snow = int(json_result['daily']['snowfall_sum'][0])              # cm        
        except:
            total_snow = None
        try:
            cloudcover_mean = round(sum(json_result['hourly']['cloudcover'])/len(json_result['hourly']['cloudcover']))     # %
        except:
            cloudcover_mean = None

        if total_rain is None and total_snow is None and cloudcover_mean is None:
            result = "no weather data"    
        elif total_rain == 0 and total_snow == 0 and cloudcover_mean < 40:
            result = "sunny"
        elif total_rain == 0 and total_snow == 0 and cloudcover_mean >= 40:
            result = "cloudy"
        elif total_rain > 0 and total_snow == 0:
            result = "rain"
        elif total_rain == 0 and total_snow > 0:
            result = "snow"
        elif total_rain > 0 and total_snow > 0:
            result = "snow & rain"
        else:
            result = f"Error, total rain: {total_rain}, total snow: {total_snow}, mean cloud: {cloudcover_mean}"

        return result  

In [21]:
latlong_df['lat'][latlong_df['city_state'] == "Denver, CO"]

18    39.762
Name: lat, dtype: float64

In [22]:
#test
weather_condition("Denver, CO", "2019-11-23", latlong_df)

'sunny'

## Run the API call on our data

In [102]:
from tqdm import tqdm           #to display the progress

In [27]:
for row in tqdm(range(len(flight_unique))):
    flight_unique['weather_categ'][row] = weather_condition(flight_unique['city'][row], flight_unique['fl_date'][row], latlong_df)

  0%|                                                                                         | 0/2450 [00:00<?, ?it/s]C:\Users\3_hal\AppData\Local\Temp\ipykernel_22196\3046711196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_unique['weather_categ'][row] = weather_condition(flight_unique['city'][row], flight_unique['fl_date'][row], latlong_df)
  0%|                                                                                 | 2/2450 [00:00<13:42,  2.98it/s]

None None Adak Island, AK


  1%|▌                                                                               | 16/2450 [00:05<13:55,  2.91it/s]

None None Ashland, WV


  1%|▊                                                                               | 25/2450 [00:08<17:04,  2.37it/s]

...

 95%|██████████████████████████████████████████████████████████████████████████▏   | 2331/2450 [13:09<00:42,  2.79it/s]

None None Newburgh/Poughkeepsie, NY


 97%|███████████████████████████████████████████████████████████████████████████▋  | 2379/2450 [13:25<00:24,  2.84it/s]

None None Saipan, TT


100%|██████████████████████████████████████████████████████████████████████████████| 2450/2450 [13:50<00:00,  2.95it/s]


In [29]:
flight_unique

,fl_date,city,count,weather_categ
758,2020-01-03,"Chicago, IL",2591,cloudy
408,2020-01-02,"Chicago, IL",2587,sunny
1463,2020-01-05,"Chicago, IL",2556,cloudy
1813,2020-01-06,"Chicago, IL",2556,sunny
2166,2020-01-07,"Chicago, IL",2413,sunny
...,...,...,...,...
1383,2020-01-04,"Vernal, UT",2,sunny
209,2020-01-01,"Moab, UT",2,cloudy
278,2020-01-01,"Salina, KS",1,sunny
183,2020-01-01,"Liberal, KS",1,sunny


In [104]:
flight_unique['weather_categ'].isna().sum()

102

In [105]:
flight_unique['weather_categ'] = flight_unique['weather_categ'].fillna('empty')

In [106]:
flight_unique.groupby(by = 'weather_categ').count()

,fl_date,city
weather_categ,,
cloudy,615,615
empty,102,102
no weather data,28,28
rain,372,372
snow,136,136
snow & rain,29,29
sunny,1168,1168


First, let's see which ones had no weather and see confirm that there was actually no weather data returned

In [74]:
flight_unique[flight_unique['weather_categ'] == 'no weather data'].groupby(by = 'city').count()

,fl_date,weather_categ
city,,
"Kahului, HI",7,7
"Kapalua, HI",7,7
"Key West, FL",7,7
"Kotzebue, AK",7,7


In [99]:
open_weather_api(20.887218, -156.482834, '2020-01-03')

{'latitude': 21.0,
 'longitude': -156.5,
 'generationtime_ms': 0.32591819763183594,
 'utc_offset_seconds': -18000,
 'timezone': 'America/Chicago',
 'timezone_abbreviation': 'CDT',
 'elevation': 0.0,
 'hourly_units': {'time': 'iso8601',
  'precipitation': 'mm',
  'rain': 'mm',
  'snowfall': 'cm',
  'cloudcover': '%'},
 'hourly': {'time': ['2020-01-03T00:00',
   '2020-01-03T01:00',
   '2020-01-03T02:00',
   '2020-01-03T03:00',
   '2020-01-03T04:00',
   '2020-01-03T05:00',
   '2020-01-03T06:00',
   '2020-01-03T07:00',
   '2020-01-03T08:00',
   '2020-01-03T09:00',
   '2020-01-03T10:00',
   '2020-01-03T11:00',
   '2020-01-03T12:00',
   '2020-01-03T13:00',
   '2020-01-03T14:00',
   '2020-01-03T15:00',
   '2020-01-03T16:00',
   '2020-01-03T17:00',
   '2020-01-03T18:00',
   '2020-01-03T19:00',
   '2020-01-03T20:00',
   '2020-01-03T21:00',
   '2020-01-03T22:00',
   '2020-01-03T23:00'],
  'precipitation': [None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   

this confirms there is no weather data for this location with this api

Second, let's see which ones returned 'empty'. We know these are likly cities that were missing lat/long values

In [107]:
flight_unique[flight_unique['weather_categ'] == 'empty'].groupby(by = 'city').count()

,fl_date,weather_categ
city,,
"Adak Island, AK",2,2
"Ashland, WV",7,7
"Barrow, AK",7,7
"Charlotte Amalie, VI",7,7
"Christiansted, VI",7,7
"Deadhorse, AK",7,7
"Devils Lake, ND",7,7
"Guam, TT",7,7
"Hilton Head, SC",7,7


Looks like we were missing the lat/longs for these 16 cities. Let's find the lat/longs and create a new lat/long df

In [83]:
latlong_dict = {'city': ['Kahului, HI', 'Kapalua, HI', 'Key West, FL','Kotzebue, AK'], 
                'lat': [20.887218, 20.9622, 24.5537, 66.898331], 
                'lng': [-156.482834, -156.6754, -81.7550, -162.596664]}
extra_latlong = pd.DataFrame(latlong_dict)

In [84]:
extra_latlong

,city,lat,lng
0,"Kahului, HI",20.887218,-156.482834
1,"Kapalua, HI",20.962200,-156.675400
2,"Key West, FL",24.553700,-81.755000
3,"Kotzebue, AK",66.898331,-162.596664


In [90]:
#rename city to city_state
extra_latlong.rename({'city': 'city_state'}, axis = 1, inplace= True)

In [91]:
extra_latlong

,city_state,lat,lng
0,"Kahului, HI",20.887218,-156.482834
1,"Kapalua, HI",20.962200,-156.675400
2,"Key West, FL",24.553700,-81.755000
3,"Kotzebue, AK",66.898331,-162.596664


In [87]:
#filter the flights df so we only have to run through these cities
extra_flights_unique = flight_unique[flight_unique['weather_categ'] == 'no weather data']

In [88]:
extra_flights_unique.head()

,fl_date,city,weather_categ
337,2020-01-03,"Kahului, HI",no weather data
338,2020-01-05,"Kahului, HI",no weather data
339,2020-01-04,"Kahului, HI",no weather data
342,2020-01-02,"Kahului, HI",no weather data
343,2020-01-01,"Kahului, HI",no weather data


In [95]:
#reset index
extra_flights_unique.reset_index(inplace = True)

In [96]:
extra_flights_unique['city'][0]

'Kahului, HI'

In [97]:
#run the function
for row in tqdm(range(len(extra_flights_unique))):
    extra_flights_unique['weather_categ'][row] = weather_condition(extra_flights_unique['city'][row], extra_flights_unique['fl_date'][row], extra_latlong)

  0%|                                                                                           | 0/28 [00:00<?, ?it/s]C:\Users\3_hal\AppData\Local\Temp\ipykernel_22196\454287497.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_flights_unique['weather_categ'][row] = weather_condition(extra_flights_unique['city'][row], extra_flights_unique['fl_date'][row], extra_latlong)
100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:10<00:00,  2.69it/s]


In [98]:
extra_flights_unique

,index,fl_date,city,weather_categ
0,337,2020-01-03,"Kahului, HI",no weather data
1,338,2020-01-05,"Kahului, HI",no weather data
2,339,2020-01-04,"Kahului, HI",no weather data
3,342,2020-01-02,"Kahului, HI",no weather data
4,343,2020-01-01,"Kahului, HI",no weather data
5,355,2020-01-06,"Kahului, HI",no weather data
6,386,2020-01-07,"Kahului, HI",no weather data
7,723,2020-01-04,"Key West, FL",no weather data
8,760,2020-01-05,"Key West, FL",no weather data
9,804,2020-01-03,"Key West, FL",no weather data


In [30]:
#save the new colummn to the csv
flight_unique.to_csv('data/flights_test_uniques.csv', header=True, index=False)

## Add the weather data for origin and dest city to flights_test df

In [52]:
#load flights_test
flights_test = pd.read_csv('data/flights_test.csv', index_col=0)

In [53]:
flights_test.head(2)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363


In [59]:
#load flights_test_unique with the weather data
flight_unique = pd.read_csv('data/flights_test_uniques.csv')

In [60]:
flight_unique.head(2)

,fl_date,city,count,weather_categ
0,2020-01-03,"Chicago, IL",2591,cloudy
1,2020-01-02,"Chicago, IL",2587,sunny


In [61]:
#we don't want to add the count, only the weather
flight_unique.drop('count', axis = 1, inplace = True)

In [55]:
#merge for origin city
flights_test = pd.merge(flights_test, flight_unique, how = 'left', left_on = ['fl_date', 'origin_city_name'], right_on = ['fl_date', 'city'])

In [56]:
flights_test.head(2)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,city,weather_categ
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,SFO,"San Francisco, CA",1810,1945,N,95,1,363,"Ontario, CA",sunny
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,...,SFO,"San Francisco, CA",1150,1320,N,90,1,363,"Ontario, CA",sunny


In [62]:
#rename col weather_origin
flights_test.rename({'weather_categ': 'orig_weather_categ'}, axis = 1, inplace= True)
#drop the city col which also merged
flights_test.drop('city', axis = 1, inplace = True)

In [63]:
flights_test.head(2)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,orig_weather_categ
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363,sunny
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,...,14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363,sunny


In [64]:
#add the dest weather
flights_test = pd.merge(flights_test, flight_unique, how = 'left', left_on = ['fl_date', 'dest_city_name'], right_on = ['fl_date', 'city'])

In [65]:
flights_test.head(2)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,orig_weather_categ,city,weather_categ
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,"San Francisco, CA",1810,1945,N,95,1,363,sunny,"San Francisco, CA",cloudy
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,...,"San Francisco, CA",1150,1320,N,90,1,363,sunny,"San Francisco, CA",cloudy


In [66]:
#rename col weather dest
flights_test.rename({'weather_categ': 'dest_weather_categ'}, axis = 1, inplace= True)
#drop the city col which also merged
flights_test.drop('city', axis = 1, inplace = True)

In [67]:
flights_test.head(2)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,orig_weather_categ,dest_weather_categ
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,SFO,"San Francisco, CA",1810,1945,N,95,1,363,sunny,cloudy
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,...,SFO,"San Francisco, CA",1150,1320,N,90,1,363,sunny,cloudy


In [70]:
flights_test[['origin_city_name','dest_city_name', 'orig_weather_categ','dest_weather_categ']]

,origin_city_name,dest_city_name,orig_weather_categ,dest_weather_categ
0,"Ontario, CA","San Francisco, CA",sunny,cloudy
1,"Ontario, CA","San Francisco, CA",sunny,cloudy
2,"Ontario, CA","San Jose, CA",sunny,cloudy
3,"Ontario, CA","San Jose, CA",sunny,cloudy
4,"Ontario, CA","San Jose, CA",sunny,cloudy
...,...,...,...,...
150618,"Detroit, MI","New York, NY",cloudy,cloudy
150619,"Greer, SC","New York, NY",rain,cloudy
150620,"Atlanta, GA","Fayetteville, AR",rain,sunny
150621,"Fayetteville, AR","Atlanta, GA",sunny,rain


In [68]:
#save over the flights_test csv
flights_test.to_csv('data/flights_test.csv', header=True, index=False)